In [7]:
%run code/import_libs.py
%run code/functions_v1.4.py
%run code/Build_Forms_v1.4.py
%run code/initial_data_prep.py

%matplotlib inline


initial_pred = df.drop(columns=["price"]).copy()
initial_price = df[["price"]]

mean_price_2014_2015=initial_price.mean()[0]

# Create OLS linear model
pred_fin, price_fin = transform_data(initial_pred, initial_price)

pred_int = sm.add_constant(pred_fin)
model = sm.OLS(price_fin,pred_int).fit()

coef_df=model.params.reset_index()
coef_df.columns=["Column","Value"]

#############################################################################
def calculate_price (sqft_living, decade, basement, zipcode, grade, waterfront, view , sqft_lot,  
                     mean_price, coef_df=coef_df, output='yes'):
 
    if waterfront == 'NO' or not waterfront: 
        waterfront = 0
    else: 
        waterfront = 1
            
    b0,b1,b2,b3,b4,b5,b6,b7,b8 = get_coeff( decade, zipcode, grade, waterfront, view, coef_df)
    y=round( np.exp(b0 + b1*np.log(sqft_living) + b2*np.log(sqft_lot) + b3*basement + b4*waterfront + b5*grade + b6 + b7 + b8) )
    
    if output == 'yes': y=y*(mean_price/mean_price_2014_2015)
    # print('{:,.0f}'.format(y))
    return y,b0,b1,b2,b3,b4,b5,b6,b7,b8
####################################################################################

In [8]:
#np.array(df[['sqft_living', 'decade', 'basement', 'zipcode','grade','waterfront','view',  'sqft_lot', 'price']].sample())[0]

model.summary()

#calculate_price(1620, 1956, 1, 98008, 7, 'YES', 'GOOD', 7875, 425000, coef_df=coef_df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     1765.
Date:                Fri, 01 Apr 2022   Prob (F-statistic):               0.00
Time:                        14:31:17   Log-Likelihood:                 5922.4
No. Observations:               21143   AIC:                        -1.167e+04
Df Residuals:                   21055   BIC:                        -1.097e+04
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             7.7842      0.035    221.453      0.000       7.715       7.853
sqft_living       0.4536      0.005     86.598      0.000       0.443       0.464
sqft_lot          0.0772      0.002     35.980      0.000       0.073       0.081
dec_1920-1939     0.0060      0.007      0.888      0.374      -0.007       0.019
dec_1940-1949    -0.0468      0.007     -6.616      0.000      -0.061      -0.033
dec_1950-1959    -0.0949      0.007    -13.979      0.000      -0.108      -0.082
dec_1960-1969    -0.1190      0.007    -17.156      0.000      -0.133      -0.105
dec_1970-1979    -0.1286      0.007    -17.750      0.000      -0.143      -0.114
dec_1980-1989    -0.1083      0.007    -14.914      0.000      -0.123      -0.094
dec_1990-1999    -0.0999      0.007    -13.489      0.000      -0.114      -0.085
dec_2000-2009    -0.0653      0.007     -9.324      0.000      -0.079      -0.052
dec_2010-2029     0.0051      0.008      0.617      0.537      -0.011       0.021
zip_98002         0.0063      0.016      0.385      0.700      -0.026       0.038
zip_98003         0.0396      0.015      2.671      0.008       0.011       0.069
zip_98004         1.1674      0.014     80.517      0.000       1.139       1.196
zip_98005         0.7872      0.017     45.187      0.000       0.753       0.821
zip_98006         0.6969      0.013     53.414      0.000       0.671       0.722
zip_98007         0.6985      0.018     37.770      0.000       0.662       0.735
zip_98008         0.6974      0.015     46.864      0.000       0.668       0.727
zip_98010         0.2415      0.021     11.486      0.000       0.200       0.283
zip_98011         0.4730      0.016     28.706      0.000       0.441       0.505
zip_98014         0.2834      0.019     14.580      0.000       0.245       0.321
zip_98019         0.3132      0.017     18.844      0.000       0.281       0.346
zip_98022         0.0468      0.016      2.977      0.003       0.016       0.078
zip_98023     -1.356e-05      0.013     -0.001      0.999      -0.025       0.025
zip_98024         0.3994      0.023     17.366      0.000       0.354       0.444
zip_98027         0.5348      0.013     39.725      0.000       0.508       0.561
zip_98028         0.4290      0.015     29.060      0.000       0.400       0.458
zip_98029         0.6278      0.014     43.451      0.000       0.599       0.656
zip_98030         0.0579      0.015      3.815      0.000       0.028       0.088
zip_98031         0.1074      0.015      7.186      0.000       0.078       0.137
zip_98032        -0.0005      0.019     -0.027      0.978      -0.038       0.037
zip_98033         0.8185      0.013     61.611      0.000       0.792       0.845
zip_98034         0.5802      0.013     45.814      0.000       0.555       0.605
zip_98038         0.1773      0.012     14.217   

In [12]:

abs_test_results=[]
test_results=[]
for i in range(1000):
    X=np.array(df[['sqft_living', 'decade', 'basement', 'zipcode','grade','waterfront','view',  'sqft_lot', 'price']].sample())[0]
    sqft_living, decade, basement, zipcode, grade, waterfront, view , sqft_lot, price=X

    try:
        y,b0,b1,b2,b3,b4,b5,b6,b7,b8=calculate_price(sqft_living, decade, basement, zipcode, grade, waterfront, 
                                                 view , sqft_lot,  550000, coef_df=coef_df, output='no')
    except: 
        print (sqft_living, decade, basement, zipcode, grade, waterfront, view , sqft_lot,  550000)
 #       continue
#    else:
#        print ("It works=====>", sqft_living, decade, basement, zipcode, grade, waterfront, view , sqft_lot,  550000)

    miss_abs= np.abs(100-(y/price*100))
    miss= 100-(y/price*100)
    test_results.append(miss)
    abs_test_results.append(miss_abs)

avg_miss=sum(test_results)/len(test_results)
avg_miss_abs=sum(abs_test_results)/len(abs_test_results)
print (avg_miss, avg_miss_abs)
#####################

-0.8123610327586887 13.913421983040077


In [10]:


#buttonW=form_items[5].children[0]

inp={ 'view':viewW,'waterfront':waterW,
    'zipcode': zipW, 'decade':decadeW, 'grade':gradeW, 'basement':basementW, "mean_price":meanW,
    'sqft_living':livingW,'sqft_lot':lotW }

output = widgets.interactive_output(calculate_price, inp )



ui = widgets.VBox([form, output])

output.layout={'border': '3px solid green', 'width':'150px'}



In [26]:
display(ui)

In [27]:
None